In [1]:
spark

In [2]:
!hadoop fs -ls /tmp

Found 3 items
-rw-r--r--   2 nileshnandan_ts hadoop    1060750 2025-08-31 05:38 /tmp/customers1mb.csv
drwxrwxrwt   - hdfs            hadoop          0 2025-08-31 04:21 /tmp/hadoop-yarn
drwx-wx-wx   - hive            hadoop          0 2025-08-31 04:22 /tmp/hive


In [5]:
!hadoop fs -head /tmp/customers1mb.csv

customer_id,name,city,state,country,registration_date,is_active
0,Customer_0,Pune,Maharashtra,India,2023-06-29,False
1,Customer_1,Bangalore,Tamil Nadu,India,2023-12-07,True
2,Customer_2,Hyderabad,Gujarat,India,2023-10-27,True
3,Customer_3,Bangalore,Karnataka,India,2023-10-17,False
4,Customer_4,Ahmedabad,Karnataka,India,2023-03-14,False
5,Customer_5,Hyderabad,Karnataka,India,2023-07-28,False
6,Customer_6,Pune,Delhi,India,2023-08-29,False
7,Customer_7,Ahmedabad,West Bengal,India,2023-12-28,True
8,Customer_8,Pune,Karnataka,India,2023-06-22,True
9,Customer_9,Mumbai,Telangana,India,2023-01-05,True
10,Customer_10,Pune,Gujarat,India,2023-08-05,True
11,Customer_11,Delhi,West Bengal,India,2023-08-02,False
12,Customer_12,Chennai,Gujarat,India,2023-11-21,False
13,Customer_13,Chennai,Karnataka,India,2023-11-06,True
14,Customer_14,Hyderabad,Tamil Nadu,India,2023-02-07,False
15,Customer_15,Mumbai,Gujarat,India,2023-03-02,True
16,Customer_16,Chennai,Karnataka,India,2023-04-05,False
17,Customer_17,Hyd

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
.appName("RBGB") \
.getOrCreate()

25/08/31 05:42:41 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [4]:
hdfs_path = '/tmp/customers1mb.csv'
rdd = spark.sparkContext.textFile(hdfs_path)

In [6]:
header = rdd.first()
data = rdd.filter(lambda x:x!=header).map(lambda y:y.split(','))

In [8]:
data.first()

['0', 'Customer_0', 'Pune', 'Maharashtra', 'India', '2023-06-29', 'False']

In [9]:
reduceby = data.map(lambda x:(x[2],1)).reduceByKey(lambda a,b:a+b)

In [10]:
reduceby.collect()

[('Pune', 2243),
 ('Hyderabad', 2242),
 ('Mumbai', 2142),
 ('Delhi', 2200),
 ('Bangalore', 2211),
 ('Ahmedabad', 2198),
 ('Chennai', 2194),
 ('Kolkata', 2223)]

In [11]:
groupby = data.map(lambda x:(x[2],1)).groupByKey()

In [13]:
groupby.collect() #((Pune),(1,1,1,1,1))

[('Pune', <pyspark.resultiterable.ResultIterable at 0x7fbc9d14dcd0>),
 ('Hyderabad', <pyspark.resultiterable.ResultIterable at 0x7fbc9f1361d0>),
 ('Mumbai', <pyspark.resultiterable.ResultIterable at 0x7fbc9d14e9d0>),
 ('Delhi', <pyspark.resultiterable.ResultIterable at 0x7fbc9d14e550>),
 ('Bangalore', <pyspark.resultiterable.ResultIterable at 0x7fbc9d14ec50>),
 ('Ahmedabad', <pyspark.resultiterable.ResultIterable at 0x7fbc9f136f50>),
 ('Chennai', <pyspark.resultiterable.ResultIterable at 0x7fbc9d14ee10>),
 ('Kolkata', <pyspark.resultiterable.ResultIterable at 0x7fbc9d14e650>)]

In [15]:
groupby_result = groupby.map(lambda r:(r[0], len(r[1])))

In [16]:
groupby_result.collect()

[('Pune', 2243),
 ('Hyderabad', 2242),
 ('Mumbai', 2142),
 ('Delhi', 2200),
 ('Bangalore', 2211),
 ('Ahmedabad', 2198),
 ('Chennai', 2194),
 ('Kolkata', 2223)]

In [17]:
#reduceby all in one

rdd.filter(lambda x:x!=header).map(lambda y:y.split(',')).map(lambda x:(x[2],1)).reduceByKey(lambda a,b:a+b).collect()

[('Pune', 2243),
 ('Hyderabad', 2242),
 ('Mumbai', 2142),
 ('Delhi', 2200),
 ('Bangalore', 2211),
 ('Ahmedabad', 2198),
 ('Chennai', 2194),
 ('Kolkata', 2223)]

In [18]:
#groupby all in one
#aggregates the data incereasing the shuffle read and shuffle write space

rdd.filter(lambda x:x!=header).map(lambda y:y.split(",")).map(lambda x:(x[2],1)).groupByKey().map(lambda r:(r[0],len(r[1]))).collect()

[('Pune', 2243),
 ('Hyderabad', 2242),
 ('Mumbai', 2142),
 ('Delhi', 2200),
 ('Bangalore', 2211),
 ('Ahmedabad', 2198),
 ('Chennai', 2194),
 ('Kolkata', 2223)]